In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV3Small, ResNet50
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, BatchNormalization
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

In [4]:
train_dir = 'dataset/split/train'
test_dir = 'dataset/split/test'

In [5]:
train_data = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="training",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=42
)

val_data = image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=42
)

test_data = image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=42
)

Found 3600 files belonging to 18 classes.
Using 2880 files for training.
Found 3600 files belonging to 18 classes.
Using 720 files for validation.
Found 414 files belonging to 18 classes.


In [6]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

In [7]:
train_features = []
train_labels = []

for batch_images, batch_labels in train_data:
    features_batch = model.predict(batch_images)
    train_features.append(features_batch)
    train_labels.append(batch_labels)

train_features = np.concatenate(train_features)
train_labels = np.concatenate(train_labels)
train_features = train_features.reshape((train_features.shape[0], -1))

1/1 [==============================] - 0s 47ms/step


In [8]:
valid_features = []
valid_labels = []

for batch_images, batch_labels in val_data:
    features_batch = model.predict(batch_images)
    valid_features.append(features_batch)
    valid_labels.append(batch_labels)

valid_features = np.concatenate(valid_features)
valid_labels = np.concatenate(valid_labels)
valid_features = valid_features.reshape((valid_features.shape[0], -1))

1/1 [==============================] - 2s 2s/step


In [9]:
test_features = []
test_labels = []

for batch_images, batch_labels in test_data:
    features_batch = model.predict(batch_images)
    test_features.append(features_batch)
    test_labels.append(batch_labels)

test_features = np.concatenate(test_features)
test_labels = np.concatenate(test_labels)
test_features = test_features.reshape((test_features.shape[0], -1))

1/1 [==============================] - 1s 966ms/step


In [10]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(train_features, np.argmax(train_labels, axis=1))
train_accuracy = rf_classifier.score(train_features, np.argmax(train_labels, axis=1))
valid_accuracy = rf_classifier.score(valid_features, np.argmax(valid_labels, axis=1))
test_accuracy = rf_classifier.score(test_features, np.argmax(test_labels, axis=1))

In [11]:
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')
print(f'Validation Accuracy: {valid_accuracy * 100:.2f}%')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Training Accuracy: 100.00%
Validation Accuracy: 69.31%
Test Accuracy: 68.60%
